<a href="https://colab.research.google.com/github/saravanamuthu1/sentiment_analysis/blob/main/sentiment_analysis_sractch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
mylist=[]
label=[]
value1=[]
testfile = open('train-v2.tsv', 'r')
for line in testfile:
  value=line.rstrip("\n").split("\t")
  mylist.append(line.rstrip("\n").split("\t"))
  label.append(value[0])
  value1.append(value[1])


In [ ]:
print(len(label))
print(len(value1))
print(len(mylist))

In [ ]:
train_set,test_set=value1[10000:80000],value1[0:10000]
train_label,test_lable=label[10000:80000],label[0:10000]

In [ ]:
def remove_word(list_to_be_removed):
  for i,j in enumerate(list_to_be_removed):
    text = j.replace("@USER", "")
    list_to_be_removed.insert(i,text)
  return list_to_be_removed




tokenize data

get the probability

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
import re

In [ ]:
df=pd.DataFrame(mylist)
df.columns=["label","tweet"]
df

In [ ]:
positive_frequency=len(df.loc[df.label == '1'])
negative_frequency=len(df.loc[df.label == "0"])
print(positive_frequency)
print(negative_frequency)
from collections import Counter

In [ ]:
df['tweet'] = df.tweet.apply(lambda x:re.sub(r'[^\w\s]', '', x))
df['tweet'] = df.tweet.apply(lambda x:x.strip())
df['tweet'] = df.tweet.apply(lambda x:x.replace("USER", ""))

In [ ]:
df['tweet'] 

In [ ]:
df['tweet'] =df.tweet.apply(lambda x:''.join(x))
df["tweet"]= df.tweet.apply(lambda x:" ".join(x.split()))
 

In [ ]:
V_freq = Counter(" ".join(df['tweet'].values.tolist()).split(" "))
print(len(V_freq))

In [ ]:
freq_pos = Counter(" ".join(df.loc[df.label=='1']['tweet'].values.tolist()).split(" ")) 

freq_neg = Counter(" ".join(df.loc[df.label=='0']['tweet'].values.tolist()).split(" ")) 


In [ ]:
print(len(freq_pos))
print(len(freq_neg))

In [ ]:
total_porobaility_for_pos=((len(freq_pos)/len(V_freq)))
print(total_porobaility_for_pos)

In [ ]:
total_porobaility_for_neg=(len(freq_neg)/len(V_freq))
print(total_porobaility_for_neg)

In [ ]:
def pos_word_loglikelihood(w):
    if w in freq_pos:
      p_w_pos=float(freq_pos.get(w,0))/len(V_freq)
      return p_w_pos
    else:
      return 1
    

In [ ]:
def neg_word_loglikelihood(w):
  if w in freq_neg:
    p_w_neg =float(freq_neg.get(w,0))/len(V_freq)
    return p_w_neg
  else:
    return 1

In [ ]:
def total_score_label(mytxt):
  mytxt=mytxt.split()
  pos_score = total_porobaility_for_pos 
  neg_score = total_porobaility_for_neg
  for w in mytxt:
      pos_score= pos_score*pos_word_loglikelihood(w)
      neg_score= neg_score*neg_word_loglikelihood(w)
  max_value=max(pos_score,neg_score)
  if max_value == pos_score:
    return 1
  else:
    return 0



In [ ]:
df['total_prediction']=df.tweet.apply(lambda x:total_score_label(x))




In [ ]:
df['label'] = df.label.apply(lambda x:int(x))

In [ ]:
accuracy = accuracy_score(df['label'], df['total_prediction'])

In [ ]:
accuracy

In [ ]:
df['total_prediction'].value_counts()

In [ ]:
df['total_prediction']